In [66]:
import numpy as np
import time
import math
import random

In [67]:
def read_input_from_file(filename):
    with open(filename, 'r') as file:
        lines = file.readlines()
    N, D, A, B = map(int, lines[0].split())
    dayoff = np.zeros((N+1, D+1))
    for i in range(1, N+1):
        numbers = list(map(int, lines[i].split()))
        for j in range(0, len(numbers) - 1):
            dayoff[i][numbers[j]] = 1
    return N, D, A, B, dayoff

In [68]:
def adjust_schedule(X, N, D, A, B, dayoff):
    # Điều chỉnh lịch để mỗi ca có từ A đến B nhân viên
    for d in range(1, D + 1):
        for shift in range(1, 5):  # Các ca 1, 2, 3, 4
            count = np.sum(X[:, d] == shift)
            while count < A:
                available = [i for i in range(1, N + 1) if X[i][d] == 0 and dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                if not available:
                    break
                i = random.choice(available)
                X[i][d] = shift
                if shift == 4 and d < D:
                    X[i][d + 1] = 0  # Nghỉ sau ca đêm
                count += 1
            while count > A:
                assigned = [i for i in range(1, N + 1) if X[i][d] == shift]
                if not assigned:
                    break
                i = random.choice(assigned)
                X[i][d] = 0
                count -= 1
    # Sửa lỗi ca đêm (ngày sau ca đêm phải nghỉ)
    for i in range(1, N + 1):
        for d in range(1, D):
            if X[i][d] == 4 and X[i][d + 1] != 0:
                X[i][d + 1] = 0

    # Đảm bảo các ca không bị thiếu
    for d in range(1, D + 1):
        for shift in range(1, 5):  # Các ca 1, 2, 3, 4
            count = np.sum(X[:, d] == shift)
            while count < A:
                available = [i for i in range(1, N + 1) if X[i][d] == 0 and dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                if not available:
                    break
                i = random.choice(available)
                X[i][d] = shift
                if shift == 4 and d < D:
                    X[i][d + 1] = 0  # Nghỉ sau ca đêm
                count += 1
            while count > B:
                assigned = [i for i in range(1, N + 1) if X[i][d] == shift]
                if not assigned:
                    break
                i = random.choice(assigned)
                X[i][d] = 0
                count -= 1
    return X

In [69]:
class SimulatedAnnealingScheduler:
    def __init__(self, N, D, A, B, dayoff, T0=5000, alpha=0.99, max_iterations=500, iterations_per_temp=50):
        self.N = N
        self.D = D
        self.A = A
        self.B = B
        self.dayoff = dayoff
        self.T0 = T0
        self.alpha = alpha
        self.max_iterations = max_iterations
        self.iterations_per_temp = iterations_per_temp
        self.best_solutions = []
        self.k = 5
        self.stagnation_counter = 0  # Đếm số lần không cải thiện
        self.stagnation_limit = 50  # Giới hạn trước khi khởi động lại

    def initialize_solution(self):
        """Khởi tạo giải pháp ban đầu"""
        X = np.zeros((self.N + 1, self.D + 1), dtype=int)
        for i in range(1, self.N + 1):
            for d in range(1, self.D + 1):
                if self.dayoff[i][d] == 1:
                    X[i][d] = 0

        for d in range(1, self.D + 1):
            for s in range(1, 5):
                available = [i for i in range(1, self.N + 1) if X[i][d] == 0 and self.dayoff[i][d] == 0 and (d == 1 or X[i][d-1] != 4)]
                np.random.shuffle(available)
                count = np.sum(X[:, d] == s)
                for i in available:
                    if count >= self.A:
                        break
                    X[i][d] = s
                    count += 1
        return X

    def evaluate_solution(self, X):
        """Đánh giá giải pháp"""
        penalty = 0
        for i in range(1, self.N + 1):
            for d in range(1, self.D + 1):
                if self.dayoff[i][d] == 1 and X[i][d] != 0:
                    penalty += 1000
        for d in range(1, self.D + 1):
            for s in range(1, 5):
                count = np.sum(X[:, d] == s)
                if count < self.A or count > self.B:
                    penalty += 1000
        for i in range(1, self.N + 1):
            for d in range(1, self.D):
                if X[i][d] == 4 and X[i][d+1] != 0:
                    penalty += 1000
        
        max_night_shifts = max(np.sum(X[i, 1:self.D + 1] == 4) for i in range(1, self.N + 1))
        return max_night_shifts + penalty

    def get_neighbor(self, X, aggressive=False):
        """Tạo giải pháp lân cận, có thể thay đổi nhiều ca nếu aggressive=True"""
        X_new = X.copy()
        if aggressive or np.random.random() < 0.3:  # Thay đổi lớn hơn nếu không cải thiện lâu
            # Hoán đổi ca của nhiều nhân viên trong một ngày
            d = np.random.randint(1, self.D + 1)
            num_swaps = np.random.randint(2, 4)  # Hoán đổi 2-4
            for _ in range(num_swaps):
                i1, i2 = np.random.choice(range(1, self.N + 1), 2, replace=False)
                if self.dayoff[i1][d] == 1 or self.dayoff[i2][d] == 1:
                    continue
                if d > 1 and (X_new[i1][d-1] == 4 or X_new[i2][d-1] == 4):
                    continue
                X_new[i1][d], X_new[i2][d] = X_new[i2][d], X_new[i1][d]

        else:
            # Thay đổi ca của một nhân viên, ưu tiên nhân viên có nhiều ca đêm
            night_shifts = [np.sum(X[i, 1:self.D + 1] == 4) for i in range(1, self.N + 1)]
            i = np.argmax(night_shifts) + 1  # Chọn nhân viên có nhiều ca đêm nhất
            d = np.random.randint(1, self.D + 1)
            if self.dayoff[i][d] == 1:
                return X_new

            new_shift = np.random.randint(0, 5)
            if d > 1 and X_new[i][d-1] == 4:
                new_shift = 0
            X_new[i][d] = new_shift
            if X_new[i][d] == 4 and d < self.D and self.dayoff[i][d+1] != 1:
                X_new[i][d+1] = 0

        #Điều chỉnh để thỏa mãn ràng buộc A, B
        for d in range(1, self.D + 1):
            for s in range(1, 5):
                count = np.sum(X_new[:, d] == s)
                while count < self.A:
                    avaiable = [i for i in range(1, self.N + 1) if X_new[i][d] == 0 and self.dayoff[i][d] == 0 and (d == 1 or X[i][d - 1] != 4)]
                    if avaiable:
                        i = np.random.choice(avaiable)
                        X_new[i][d] = s
                        count += 1
                    else:
                        break
                while count > self.B:
                    avaiable = [i for i in range(1, self.N + 1) if X_new[i][d] == s]
                    if avaiable:
                        i = np.random.choice(avaiable)
                        X_new[i][d] = 0
                        count -= 1
                    else:
                        break
        return X_new

    def run(self):
        """Chạy thuật toán SA"""
        X = self.initialize_solution()
        cost = self.evaluate_solution(X)
        best_X = X.copy()
        best_cost = cost
        T = self.T0

        for iteration in range(self.max_iterations):
            print(f"Iteration: {iteration} / {self.max_iterations}   and best_cost = {best_cost}")
            for _ in range(self.iterations_per_temp):
                # Tạo giải pháp lân cận, thay đổi mạnh nếu không cải thiện lâu
                aggressive = self.stagnation_counter > self.stagnation_limit
                X_new = self.get_neighbor(X, aggressive=aggressive)
                new_cost = self.evaluate_solution(X_new)
                delta_E = new_cost - cost

                if delta_E <= 0 or np.random.random() < math.exp(-delta_E / T):
                    X = X_new
                    cost = new_cost
                    self.stagnation_counter = 0  # Reset nếu có thay đổi
                else:
                    self.stagnation_counter += 1

                if cost < best_cost:
                    best_cost = cost
                    best_X = X.copy()
                    self.best_solutions.append((best_X.copy(), best_cost))
                    self.best_solutions = sorted(self.best_solutions, key=lambda x: x[1])[:self.k]

            T *= self.alpha
            if T < 0.01:
                break

        return best_X, best_cost

In [70]:
if __name__ == "__main__":
    list_test = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
    filename = 'Testcase/tc'
    output = 'Output/output'
    for tc in list_test:
        print(f"Processing tescase {tc}")
        start_time = time.time()
        input_file = f"{filename}{tc}.txt"
        output_file = f"{output}{tc}.txt"

        N, D, A, B, dayoff = read_input_from_file(input_file)
        scheduler = SimulatedAnnealingScheduler(N, D, A, B, dayoff)
        best_solution, best_cost = scheduler.run()
        end_time = time.time()
        with open(output_file, 'w') as file:
            for i in range(1, N + 1):
                file.write(' '.join([str(int(best_solution[i][d])) for d in range(1, D + 1)]) + '\n')
            end_time = time.time()
            file.write("Execution time: {:.2f} seconds\n".format(end_time - start_time))
        
        print(f"Test case {tc} completed in {end_time - start_time:.2f} seconds.")
        print(f"Bestcost = {best_cost}")
        ngay_vi_pham = []
        for i in range(1, N + 1):
            for d in range(1, D):
                if best_solution[i][d] == 4 and best_solution[i][d + 1] != 0:
                    print(f"Nhân viên {i} vi phạm quy tắc ca đêm tại ngày {d}")

        for d in range(1, D + 1):
            for shift in range(1, 5):  # Các ca 1, 2, 3, 4
                count = np.sum(best_solution[:, d] == shift)
                if count < A or count > B:
                    print(f"Vi phạm quy tắc số lượng nhân viên cho ca {shift} tại ngày {d}")
                    print (f"Số lượng nhân viên: {count}, yêu cầu: {A}-{B}")
                    ngay_vi_pham.append(d)

        for i in range(1, N + 1):
            for d in range(1, D + 1):
                if best_solution[i][d] != 0 and dayoff[i][d] == 1:
                    print(f"Nhân viên {i} vi phạm quy tắc ngày nghỉ tại ngày {d}")

        for d in ngay_vi_pham:
            print(f"Số nhân viên nghỉ tại ngày {d}:")
            print(np.sum(best_solution[:, d] == 0))

        

Processing tescase 1
Iteration: 0 / 500   and best_cost = 2
Iteration: 1 / 500   and best_cost = 1
Iteration: 2 / 500   and best_cost = 1
Iteration: 3 / 500   and best_cost = 1
Iteration: 4 / 500   and best_cost = 1
Iteration: 5 / 500   and best_cost = 1
Iteration: 6 / 500   and best_cost = 1
Iteration: 7 / 500   and best_cost = 1
Iteration: 8 / 500   and best_cost = 1
Iteration: 9 / 500   and best_cost = 1
Iteration: 10 / 500   and best_cost = 1
Iteration: 11 / 500   and best_cost = 1
Iteration: 12 / 500   and best_cost = 1
Iteration: 13 / 500   and best_cost = 1
Iteration: 14 / 500   and best_cost = 1
Iteration: 15 / 500   and best_cost = 1
Iteration: 16 / 500   and best_cost = 1
Iteration: 17 / 500   and best_cost = 1
Iteration: 18 / 500   and best_cost = 1
Iteration: 19 / 500   and best_cost = 1
Iteration: 20 / 500   and best_cost = 1
Iteration: 21 / 500   and best_cost = 1
Iteration: 22 / 500   and best_cost = 1
Iteration: 23 / 500   and best_cost = 1
Iteration: 24 / 500   and bes